In [1]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook()

Loading BokehJS ...

In [2]:
postflash_data = pd.read_pickle('all_raw_postflash_data.pkl')

In [3]:
postflash_data[0:10]

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time
rootname,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_raw.fits,13078,mastDownload/HST/ic6de2u9q/ic6de2u9q_raw.fits,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5
idp530aiq,idp530aiq_raw.fits,14984,mastDownload/HST/idp530aiq/idp530aiq_raw.fits,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5
icrz19f7q,icrz19f7q_raw.fits,14006,mastDownload/HST/icrz19f7q/icrz19f7q_raw.fits,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5
idus35y6q,idus35y6q_raw.fits,15573,mastDownload/HST/idus35y6q/idus35y6q_raw.fits,2019-08-15,10:09:26,A,7310,100.0,MED,False,0.5
idp531a7q,idp531a7q_raw.fits,14984,mastDownload/HST/idp531a7q/idp531a7q_raw.fits,2018-07-20,04:24:43,B,12,4.7,LOW,True,0.5
idus10qfq,idus10qfq_raw.fits,15573,mastDownload/HST/idus10qfq/idus10qfq_raw.fits,2019-01-02,18:39:03,B,7310,100.0,MED,False,0.5
idp503bnq,idp503bnq_raw.fits,14984,mastDownload/HST/idp503bnq/idp503bnq_raw.fits,2017-11-09,07:58:55,A,22,0.3,MED,True,0.5
id0o09n2q,id0o09n2q_raw.fits,14372,mastDownload/HST/id0o09n2q/id0o09n2q_raw.fits,2016-01-06,04:39:31,A,7310,100.0,MED,False,0.5
idus29bxq,idus29bxq_raw.fits,15573,mastDownload/HST/idus29bxq/idus29bxq_raw.fits,2019-06-21,23:40:09,B,7310,100.0,MED,False,0.5


In [4]:
low_sub_pf = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True)] 

In [5]:
subarray_pf = postflash_data.loc[(postflash_data['subarray'] == True)] 
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 

In [6]:
low_sub_pf_A = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A')] 
low_sub_pf_B = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B')] 

In [7]:
paths = subarray_pf.path.tolist()
print (paths)

['mastDownload/HST/ic6de2u9q/ic6de2u9q_raw.fits', 'mastDownload/HST/idp530aiq/idp530aiq_raw.fits', 'mastDownload/HST/icrz19f7q/icrz19f7q_raw.fits', 'mastDownload/HST/idp531a7q/idp531a7q_raw.fits', 'mastDownload/HST/idp503bnq/idp503bnq_raw.fits', 'mastDownload/HST/ie1b42krq/ie1b42krq_raw.fits', 'mastDownload/HST/id0o22kjq/id0o22kjq_raw.fits', 'mastDownload/HST/ic6dh3ioq/ic6dh3ioq_raw.fits', 'mastDownload/HST/idp518a7q/idp518a7q_raw.fits', 'mastDownload/HST/ic6dg3l1q/ic6dg3l1q_raw.fits', 'mastDownload/HST/ie1b30yeq/ie1b30yeq_raw.fits', 'mastDownload/HST/idus20j7q/idus20j7q_raw.fits', 'mastDownload/HST/idus30blq/idus30blq_raw.fits', 'mastDownload/HST/idus07oaq/idus07oaq_raw.fits', 'mastDownload/HST/idus10qdq/idus10qdq_raw.fits', 'mastDownload/HST/ic6dk2m5q/ic6dk2m5q_raw.fits', 'mastDownload/HST/ic6dh3iuq/ic6dh3iuq_raw.fits', 'mastDownload/HST/idus31qyq/idus31qyq_raw.fits', 'mastDownload/HST/id0o36eiq/id0o36eiq_raw.fits', 'mastDownload/HST/ic6dj3w5q/ic6dj3w5q_raw.fits', 'mastDownload/HST/i

In [8]:
means = []
sig_means = []
medians = []
sig_medians = []
stds = []
sig_stds = []

for f in paths:
    path = f
    print(path)
    data = fits.getdata(path)
    data_clip = stats.sigma_clip(data, 3)
    
    mean = np.mean(data)
    sig_mean = np.mean(data_clip)
    median = np.median(data)
    sig_median = np.median(data_clip)
    std = np.std(data)
    sig_std = np.std(data_clip)
    
    
    means.append(mean)
    sig_means.append(sig_mean)
    medians.append(median)
    sig_medians.append(sig_median)
    stds.append(std)
    sig_stds.append(sig_std)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means,'median': medians, 'sig_median': sig_medians, 'std': stds, 'sig_std': sig_stds}
path_and_stats_sub = pd.DataFrame(dict)



mastDownload/HST/ic6de2u9q/ic6de2u9q_raw.fits


/Users/cmartlin/miniconda3/envs/postflash_work/lib/python3.8/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


mastDownload/HST/idp530aiq/idp530aiq_raw.fits


/Users/cmartlin/miniconda3/envs/postflash_work/lib/python3.8/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


mastDownload/HST/icrz19f7q/icrz19f7q_raw.fits


...

/Users/cmartlin/miniconda3/envs/postflash_work/lib/python3.8/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


mastDownload/HST/ic6de3riq/ic6de3riq_raw.fits


/Users/cmartlin/miniconda3/envs/postflash_work/lib/python3.8/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


mastDownload/HST/id0o07hvq/id0o07hvq_raw.fits


/Users/cmartlin/miniconda3/envs/postflash_work/lib/python3.8/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [31]:
subarray_mean = pd.merge(left=subarray_pf, right=path_and_mean_sub, left_on='path', right_on='path').set_index(subarray_pf.index)

In [32]:
subarray_mean.to_pickle('all_sub_pf_and_mean_data.pkl')
#subarray_mean = pd.read_pickle('all_sub_pf_and_mean_data.pkl')
print (subarray_mean)

                     filename  propid  \
rootname                                
ic6de2u9q  ic6de2u9q_flc.fits   13078   
idp530aiq  idp530aiq_flc.fits   14984   
icrz19f7q  icrz19f7q_flc.fits   14006   
idp531a7q  idp531a7q_flc.fits   14984   
idp503bnq  idp503bnq_flc.fits   14984   
...                       ...     ...   
idus42lzq  idus42lzq_flc.fits   15573   
idp524egq  idp524egq_flc.fits   14984   
icrz24ubq  icrz24ubq_flc.fits   14006   
ic6de3riq  ic6de3riq_flc.fits   13078   
id0o07hvq  id0o07hvq_flc.fits   14372   

                                                    path     obsdate  \
rootname                                                               
ic6de2u9q  mastDownload/HST/ic6de2u9q/ic6de2u9q_flc.fits  2013-03-09   
idp530aiq  mastDownload/HST/idp530aiq/idp530aiq_flc.fits  2018-06-22   
icrz19f7q  mastDownload/HST/icrz19f7q/icrz19f7q_flc.fits  2015-05-25   
idp531a7q  mastDownload/HST/idp531a7q/idp531a7q_flc.fits  2018-07-20   
idp503bnq  mastDownload/HST/idp503

In [56]:
low_sub_pf_A_lvl_3 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'A') & (subarray_mean['flash_lvl'] == 3)]
low_sub_pf_A_lvl_3  = low_sub_pf_A_lvl_3[~low_sub_pf_A_lvl_3.obsdate.str.contains("2021")]
low_sub_pf_A_lvl_3  = low_sub_pf_A_lvl_3[~low_sub_pf_A_lvl_3.obsdate.str.contains("2020-08")]
low_sub_pf_A_lvl_3  = low_sub_pf_A_lvl_3[~low_sub_pf_A_lvl_3.obsdate.str.contains("2020-09")]
low_sub_pf_A_lvl_3  = low_sub_pf_A_lvl_3 [~low_sub_pf_A_lvl_3.obsdate.str.contains("2020-10")]
low_sub_pf_A_lvl_3  = low_sub_pf_A_lvl_3 [~low_sub_pf_A_lvl_3.obsdate.str.contains("2020-11")]
low_sub_pf_A_lvl_3  = low_sub_pf_A_lvl_3 [~low_sub_pf_A_lvl_3.obsdate.str.contains("2020-12")]

print(low_sub_pf_A_lvl_3)



                     filename  propid  \
rootname                                
icrz24uhq  icrz24uhq_flc.fits   14006   
idcd42hkq  idcd42hkq_flc.fits   14535   
idcd07o5q  idcd07o5q_flc.fits   14535   
ic6dl3q1q  ic6dl3q1q_flc.fits   13078   
ic6dj3w7q  ic6dj3w7q_flc.fits   13078   
...                       ...     ...   
id0o07hjq  id0o07hjq_flc.fits   14372   
icrz15r3q  icrz15r3q_flc.fits   14006   
idus12phq  idus12phq_flc.fits   15573   
icrz06isq  icrz06isq_flc.fits   14006   
idcd08uqq  idcd08uqq_flc.fits   14535   

                                                    path     obsdate  \
rootname                                                               
icrz24uhq  mastDownload/HST/icrz24uhq/icrz24uhq_flc.fits  2015-06-23   
idcd42hkq  mastDownload/HST/idcd42hkq/idcd42hkq_flc.fits  2017-10-10   
idcd07o5q  mastDownload/HST/idcd07o5q/idcd07o5q_flc.fits  2016-12-07   
ic6dl3q1q  mastDownload/HST/ic6dl3q1q/ic6dl3q1q_flc.fits  2013-09-21   
ic6dj3w7q  mastDownload/HST/ic6dj3

In [57]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_3['obsdate'] + ' ' + low_sub_pf_A_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_3['sig_mean'])
y = low_sub_pf_A_lvl_3['sig_mean']/low_sub_pf_A_lvl_3['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 3 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

-1.6968277310002921e-06


In [58]:
low_sub_pf_B_lvl_3 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'B') & (subarray_mean['flash_lvl'] == 3)]

low_sub_pf_B_lvl_3  = low_sub_pf_B_lvl_3[~low_sub_pf_B_lvl_3.obsdate.str.contains("2021")]
low_sub_pf_B_lvl_3  = low_sub_pf_B_lvl_3[~low_sub_pf_B_lvl_3.obsdate.str.contains("2020-08")]
low_sub_pf_B_lvl_3  = low_sub_pf_B_lvl_3[~low_sub_pf_B_lvl_3.obsdate.str.contains("2020-09")]
low_sub_pf_B_lvl_3  = low_sub_pf_B_lvl_3 [~low_sub_pf_B_lvl_3.obsdate.str.contains("2020-10")]
low_sub_pf_B_lvl_3  = low_sub_pf_B_lvl_3 [~low_sub_pf_B_lvl_3.obsdate.str.contains("2020-11")]
low_sub_pf_B_lvl_3  = low_sub_pf_B_lvl_3 [~low_sub_pf_B_lvl_3.obsdate.str.contains("2020-12")]



In [59]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_3['obsdate'] + ' ' + low_sub_pf_B_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_3['sig_mean'])
y = low_sub_pf_B_lvl_3['sig_mean']/low_sub_pf_B_lvl_3['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 3 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -5.930349604386113e-06


In [ ]:
low_sub_pf_A_lvl_5 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'A') & (subarray_mean['flash_lvl'] == 5)]

# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_5['obsdate'] + ' ' + low_sub_pf_A_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_5['sig_mean'])
y = low_sub_pf_A_lvl_5['sig_mean']/low_sub_pf_A_lvl_5['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 5 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
low_sub_pf_B_lvl_5 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'B') & (subarray_mean['flash_lvl'] == 5)]
low_sub_pf_A_lvl_8 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'A') & (subarray_mean['flash_lvl'] == 8)]
low_sub_pf_B_lvl_8 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'B') & (subarray_mean['flash_lvl'] == 8)]
low_sub_pf_A_lvl_10 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'A') & (subarray_mean['flash_lvl'] == 10)]
low_sub_pf_B_lvl_10 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'B') & (subarray_mean['flash_lvl'] == 10)]
low_sub_pf_A_lvl_12 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'A') & (subarray_mean['flash_lvl'] == 12)]
low_sub_pf_B_lvl_12 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'B') & (subarray_mean['flash_lvl'] == 12)]
low_sub_pf_A_lvl_20 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'A') & (subarray_mean['flash_lvl'] == 20)]
low_sub_pf_B_lvl_20 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['subarray'] == True) & (subarray_mean['shutter'] == 'B') & (subarray_mean['flash_lvl'] == 20)]


In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_5['obsdate'] + ' ' + low_sub_pf_B_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_5['sig_mean'])
y = low_sub_pf_B_lvl_5['sig_mean']/low_sub_pf_B_lvl_5['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 5 Shutter B")
p.line(x_datetime,y_pred,color='red', legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_8['obsdate'] + ' ' + low_sub_pf_A_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_8['sig_mean'])
y = low_sub_pf_A_lvl_8['sig_mean']/low_sub_pf_A_lvl_8['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 8 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_8['obsdate'] + ' ' + low_sub_pf_B_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_8['sig_mean'])
y = low_sub_pf_B_lvl_8['sig_mean']/low_sub_pf_B_lvl_8['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 8 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_10['obsdate'] + ' ' + low_sub_pf_A_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_10['sig_mean'])
y = low_sub_pf_A_lvl_10['sig_mean']/low_sub_pf_A_lvl_10['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 10 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_10['obsdate'] + ' ' + low_sub_pf_B_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_10['sig_mean'])
y = low_sub_pf_B_lvl_10['sig_mean']/low_sub_pf_B_lvl_10['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 10 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_12['obsdate'] + ' ' + low_sub_pf_A_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_12['sig_mean'])
y = low_sub_pf_A_lvl_12['sig_mean']/low_sub_pf_A_lvl_12['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 12 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_12['obsdate'] + ' ' + low_sub_pf_B_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_12['sig_mean'])
y = low_sub_pf_B_lvl_12['sig_mean']/low_sub_pf_B_lvl_12['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.1)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 12 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_20['obsdate'] + ' ' + low_sub_pf_A_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_20['sig_mean'])
y = low_sub_pf_A_lvl_20['sig_mean']/low_sub_pf_A_lvl_20['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 20 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_20['obsdate'] + ' ' + low_sub_pf_B_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_20['sig_mean'])
y = low_sub_pf_B_lvl_20['sig_mean']/low_sub_pf_B_lvl_20['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 20 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

"The data in Figure 10 are from program 13078 visits A3, A6, A7, and A8 and taken on November 15
and 16, 2012. Images were combined in pairs with STSDAS CRREJ, and the mean pixel value was
computed for the 1024x1024 pixel area nearest the C amplifier using IMSTAT with 3 iterations of 3
sigma rejection. "

In [ ]:
subarray_mean = pd.read_pickle('all_sub_pf_and_mean_data.pkl')
low_sub_pf_isr_13078_nov15 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['obsdate'] == '2012-11-15')]
low_sub_pf_isr_13078_nov16 = subarray_mean.loc[(subarray_mean['flash_cur'] == 'LOW') & (subarray_mean['propid'] == 13078) & (subarray_mean['obsdate'] == '2012-11-16')]

both_dates = [low_sub_pf_isr_13078_nov15, low_sub_pf_isr_13078_nov16]

low_sub_pf_isr_13078 = pd.concat(both_dates)
print(low_sub_pf_isr_13078)

In [ ]:
paths = low_sub_pf_isr_13078.path.tolist() 

means = []
sig_means = []
for f in paths:
    path = f
    data = fits.getdata(path)
    data_clip_1 = stats.sigma_clip(data, 3)
    data_clip_2 = stats.sigma_clip(data_clip_1, 3)
    data_clip_3 = stats.sigma_clip(data_clip_2, 3)
    mean = np.mean(data)
    sig_mean = np.mean(data_clip_3)
    means.append(mean)
    sig_means.append(sig_mean)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means}
path_and_mean_sub = pd.DataFrame(dict)

In [ ]:
low_sub_pf_isr_13078['count_rate'] = (low_sub_pf_isr_13078['sig_mean']/low_sub_pf_isr_13078['flash_dur']) * low_sub_pf_isr_13078['flash_dur']

In [ ]:
low_sub_pf_isr_13078

In [ ]:
med_full_pf = postflash_data.loc[(postflash_data['flash_cur'] == 'MED') & (postflash_data['subarray'] == False)] 

In [ ]:
med_full_pf